In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data80164


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

test.txt  train.txt


In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install paddleseg

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/69/bf/f0f194d3379d3f3347478bd267f754fc68c11cbf2fe302a6ab69447b1417/beautifulsoup4-4.10.0-py3-none-any.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/36/69/d82d04022f02733bf9a72bc3b96332d360c0c5307096d76f6bb7489f7e57/soupsieve-2.2.1-py3-none-any.whl
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import paddle
from paddle.io import Dataset
from paddle.vision.transforms import Compose, Transpose, ColorJitter, vflip, Grayscale, hflip, Normalize
from PIL import Image
import numpy as np
import random
import os


# 读取数据集
class Dataloader(Dataset):
    def __init__(self, model=''):
        super(Dataloader).__init__()
        self.image_floader = ''
        if model == 'train':
            self.file = 'work/train.txt'
        else:
            self.file = 'work/test.txt'
        self.jpg_list, self.label_list= self.read_list()

    def read_list(self):
        data_list = []
        jpg_list = []
        label_list = []
        with open(self.file) as lines:
            for line in lines:
                jpg_path = os.path.join("img_train", line.split(',')[0])
                label_path = os.path.join("lab_train", line.split(',')[1].replace('\n', ''))
                data_list.append((jpg_path, label_path))
        random.shuffle(data_list)
        for k in data_list:
            jpg_list.append(k[0])
            label_list.append(k[1])
        return jpg_list, label_list

    def _load_img(self, jpg_path, label_path):
        jpg = np.array(Image.open(jpg_path))
        jpg = 1 / (1 + np.exp(-((jpg - 127.5) / 127.5)))
        label = Image.open(label_path)
        return Compose([Transpose()])(jpg), Compose([Grayscale(), Transpose()])(label)

    def __getitem__(self, idx):
        train_image, label_image= self._load_img(self.jpg_list[idx], self.label_list[idx])
        train_image = np.array(train_image, dtype='float32')
        label_image = np.array(label_image, dtype='int64')
        label_image[label_image>4]=4
        zero_image=np.zeros(shape=label_image.shape)
        zero_image[label_image!=4]=1
        data=2-np.sum(zero_image)/256/256
        label_image=np.concatenate([label_image,data*zero_image])
        return train_image, label_image

    def __len__(self):
        return len(self.label_list)

# 计算损失函数
class LOSS_CROSS_IOU(paddle.nn.Layer):
    def __init__(self, weights, num_class):
        super(LOSS_CROSS_IOU, self).__init__()
        self.weights_list = weights
        self.num_class = num_class

    def forward(self, input, label):
        if(len(input)>1):
            input_1 = paddle.transpose(input[1], [0, 2, 3, 1])
            input = paddle.transpose(input[0], [0, 2, 3, 1])
            label_1 = paddle.cast(paddle.transpose(paddle.unsqueeze(label[:,0,:,:],axis=1), [0, 2, 3, 1]),dtype="int64")
            iou_loss = paddle.abs(paddle.mean(paddle.nn.functional.dice_loss(input,label_1)))
            cross_loss = paddle.mean(paddle.nn.functional.softmax_with_cross_entropy(logits=input, label=label_1))+paddle.mean(paddle.nn.functional.softmax_with_cross_entropy(logits=input_1, label=label_1))
        else:
            input = paddle.transpose(input[0], [0, 2, 3, 1])
            label_1 = paddle.cast(paddle.transpose(paddle.unsqueeze(label[:,0,:,:],axis=1), [0, 2, 3, 1]),dtype="int64")
            iou_loss = paddle.abs(paddle.mean(paddle.nn.functional.dice_loss(input,label_1)))
            cross_loss = paddle.mean(paddle.nn.functional.softmax_with_cross_entropy(logits=input, label=label_1))
        return paddle.add(iou_loss, cross_loss)

def train():
    from paddleseg.models.backbones.hrnet import HRNet_W48
    from paddleseg.models.ocrnet import OCRNet
    from paddleseg.models.backbones import ResNet101_vd
    from paddleseg.models.deeplab import DeepLabV3P
    NET=OCRNet(5,HRNet_W48(pretrained='https://bj.bcebos.com/paddleseg/dygraph/hrnet_w48_ssld.tar.gz'),(0,))
    # BackBone=ResNet101_vd()
    # NET = DeepLabV3P(5,backbone=BackBone,pretrained="https://bj.bcebos.com/paddleseg/dygraph/resnet101_vd_ssld.tar.gz")

    epoch=20
    batch_size = 48
    step = 0
    step2 = 0
    PrintNum = 20
    modelDir = "Model/"
    # layer_state_dict = paddle.load("Model/3/model.pdparams")
    # NET.set_state_dict(layer_state_dict)
    ans_list=[]
    NET.train()

    # 训练测试数据集
    train_dataset = Dataloader(model='train')
    val_dataset = Dataloader(model='test')
    train_loader = paddle.io.DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True)
    val_loader = paddle.io.DataLoader(val_dataset, batch_size=batch_size, drop_last=True, shuffle=True)

    # 设置学习率，设置优化器
    scheduler = paddle.optimizer.lr.StepDecay(learning_rate=0.01, step_size=2, gamma=0.8, verbose=True)
    sgd = paddle.optimizer.SGD(learning_rate=scheduler, parameters=NET.parameters())
    loss_fn = LOSS_CROSS_IOU(0, 5)
    for i in range(epoch):
        losses = []
        # 按Batch循环
        for batch_id, data in enumerate(train_loader()):
            step += 1
            x_data = data[0]  # 训练数据
            y_data = data[1]
            predicts = NET(x_data)
            loss = loss_fn(predicts, y_data)
            losses.append(loss.numpy())
            if batch_id % PrintNum == 0:
                print('AFTER ', i + 1, ' epochs', batch_id + 1, ' batch iou:', sum(losses) / len(losses))
            loss.backward()
            sgd.step()
            # 梯度清零
            sgd.clear_grad()
        scheduler.step()
        print('epoch iou:', sum(losses) / len(losses))
        NET.eval()
        with paddle.no_grad():
            aiou = []
            for batch_id, data in enumerate(val_loader()):
                step2 += 1
                x_data = data[0]  # 训练数据
                y_data = data[1]
                predicts = NET(x_data)
                loss = loss_fn(predicts, y_data)
                aiou.append(loss.numpy())
                if batch_id % PrintNum == 0:
                    print('test biou ', sum(aiou) / len(aiou))
            print('test biou all', sum(aiou) / len(aiou))
            ans_list.append(sum(aiou) / len(aiou))
        NET.train()
        if ans_list[-1] == min(ans_list):
            model_path = modelDir + str(i)
            paddle.save(NET.state_dict(), os.path.join(model_path, 'model.pdparams'))
            paddle.save(sgd.state_dict(), os.path.join(model_path, 'model.pdopt'))
        else:
            print(ans_list[-1], min(ans_list), 'no save')
    return 0

if __name__ == '__main__':
    train()

2021-09-24 19:30:17 [INFO]	Loading pretrained model from https://bj.bcebos.com/paddleseg/dygraph/hrnet_w48_ssld.tar.gz
2021-09-24 19:30:21 [INFO]	There are 1525/1525 variables loaded into HRNet.
Epoch 0: StepDecay set learning rate to 0.01.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:706: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:69: DeprecationWarning: 
API "paddle.nn.functional.loss.softmax_with_cross_entropy" is deprecated since 2.0.0, and will be removed in future versions. Please use "paddle.nn.functional

AFTER  1  epochs 1  batch iou: [4.1780643]


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
import paddle
from paddle.io import Dataset
from paddle.vision.transforms import Compose, Transpose, ColorJitter, vflip, Grayscale, hflip, Normalize
from PIL import Image
import numpy as np
from tqdm import tqdm
import os
from paddleseg.models.backbones.hrnet import HRNet_W48
from paddleseg.models.ocrnet import OCRNet

def predict(model_path="Model/5/model.pdparams",pic_dir="img_testA",save_dir="ans"):
    files=os.listdir(pic_dir)
    model = OCRNet(5, HRNet_W48(pretrained=None), (0,))
    layer_state_dict = paddle.load(model_path)
    model.set_state_dict(layer_state_dict)
    model.eval()
    for i in tqdm(files):
        path=os.path.join(pic_dir,i)
        data=np.array(Image.open(path))
        data = 1 / (1 + np.exp(-((data - 127.5) / 127.5)))
        data=np.reshape(data,[1,256,256,3])
        pic_data = paddle.transpose(paddle.cast(paddle.to_tensor(data), dtype='float32'), [0, 3, 1, 2])
        predicts = model(pic_data)
        pic = paddle.transpose(predicts[0], [0, 2, 3, 1])
        ans = paddle.argmax(pic, axis=-1)
        ans=ans.numpy()
        ans=np.reshape(ans,[256,256])
        im=Image.fromarray(ans.astype("uint8"))
        im.save(os.path.join(save_dir,i.replace("jpg","png")))
    return 

def change(pic_dir="img_testA"):
    files=os.listdir(pic_dir)
    for i in tqdm(files):
        path=os.path.join(pic_dir,i)
        data=np.array(Image.open(path))
        data[data>3]=1
        im=Image.fromarray(data.astype("uint8"))
        im.save(path)


if __name__ == '__main__':
    # change()
    # predict()

100%|██████████| 4608/4608 [00:12<00:00, 372.84it/s]
